<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [4]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE', 'BANKBARODA',
       'BERGEPAINT', 'BHARTIARTL', 'BRITANNIA', 'CANBK', 'CIPLA',
       'COLPAL', 'DABUR', 'DEEPAKNTR', 'DIVISLAB', 'DMART', 'DRREDDY',
       'EICHERMOT', 'FEDERALBNK', 'GODREJCP', 'GODREJIND', 'HAVELLS',
       'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO',
       'HINDALCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK', 'ICICIPRULI',
       'INDIANB', 'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTIM', 'M&M', 'MARUTI',
       'MCDOWELL-N', 'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM', 'NAUKRI',
       'PIDILITIND', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN', 'SUNPHARMA',
       'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL', 'TCS',
       'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO', 'ZEEL'], dtype=object)

In [5]:
mypf['InPortfolio'].value_counts()

0    54
1    10
Name: InPortfolio, dtype: int64

In [6]:
mypf[mypf['InPortfolio'] == 1]['CCIFlag'].value_counts()

0.0    7
1.0    3
Name: CCIFlag, dtype: int64

In [7]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [8]:
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"


  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [9]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='1y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    #stock_df.ta.macd(close='Close', fast=12, slow=26, append=True)
    #stock_df['Trend'] = 'Down'
    #stock_df.loc[stock_df['MACD_12_26_9'] - stock_df['MACDs_12_26_9'] > 10, 'Trend'] = 'Up'
    #stock_df.ta.eom(append=True)    
    stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'EOM_14_100000000':'EOM'}, inplace=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0) 
    #stock_df['EOM'] = round(stock_df['EOM'],0) 
    #stock_df['EOM'] = '-ve'
    #stock_df.loc[stock_df['EOM_14_100000000'] > 0, 'EOM'] = '+ve'    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   #'MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9','EOM_14_100000000'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
    tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [10]:
stock_prec_dev('^NSEI').tail(5)

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI
Date,,,,,,,,,,
2022-12-26 00:00:00+05:30,18015.0,^NSEI,4.76,0.48,0.53,18812.0,15294.0,77.33,22.67,41.0
2022-12-27 00:00:00+05:30,18132.0,^NSEI,5.38,0.50,0.52,18812.0,15294.0,80.68,19.32,45.0
2022-12-28 00:00:00+05:30,18122.0,^NSEI,5.27,0.51,0.50,18812.0,15294.0,80.40,19.60,45.0
2022-12-29 00:00:00+05:30,18191.0,^NSEI,5.62,0.52,0.45,18812.0,15294.0,82.35,17.65,47.0
2022-12-30 00:00:00+05:30,18105.0,^NSEI,5.07,0.50,0.44,18812.0,15294.0,79.91,20.09,44.0


In [11]:
# MACD Uptrend
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close'])*100/(df_prec_dev['Close']),0)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
#df_prec_dev[df_prec_dev['Trend'] == 'Up'].sort_values(by = 'Upside%', ascending=False)

In [12]:
# portfolio stocks
mypf['Stock'] = mypf['Symbol'] + '.NS'
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 10


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
1,2022-12-30 00:00:00+05:30,451.0,ICICIPRULI.NS,-12.54,-0.19,-1.26,615.0,440.0,6.46,93.54,43.0,640.0,60.9,75.8,42.0
2,2022-12-30 00:00:00+05:30,686.0,MOTILALOFS.NS,-10.86,-0.91,-0.15,960.0,654.0,10.57,89.43,50.0,930.0,19.7,9.4,36.0
3,2022-12-30 00:00:00+05:30,1231.0,SBILIFE.NS,3.21,0.75,0.32,1332.0,1023.0,67.41,32.59,44.0,1607.0,62.1,1000.0,31.0
4,2022-12-30 00:00:00+05:30,388.0,TATAMOTORS.NS,-9.36,-0.23,-0.24,525.0,372.0,10.42,89.58,36.0,495.0,19.2,1000.0,28.0
10,2022-12-30 00:00:00+05:30,566.0,HDFCLIFE.NS,2.03,0.36,0.71,672.0,502.0,37.79,62.21,46.0,681.0,89.3,82.8,20.0
12,2022-12-30 00:00:00+05:30,3932.0,NAUKRI.NS,-3.20,-0.57,0.30,5667.0,3363.0,24.70,75.30,44.0,4677.0,78.8,17.1,19.0
18,2022-12-30 00:00:00+05:30,1063.0,MUTHOOTFIN.NS,-4.14,-1.26,0.37,1517.0,956.0,19.08,80.92,45.0,1217.0,12.5,11.4,14.0
21,2022-12-30 00:00:00+05:30,1508.0,INFY.NS,-1.49,-0.65,0.32,1898.0,1351.0,28.74,71.26,41.0,1715.0,23.0,27.7,14.0
29,2022-12-30 00:00:00+05:30,1016.0,TECHM.NS,-7.52,-1.73,0.10,1744.0,935.0,10.06,89.94,44.0,1139.0,18.7,18.7,12.0
51,2022-12-30 00:00:00+05:30,3413.0,DIVISLAB.NS,-9.13,-0.96,-0.41,4643.0,3266.0,10.69,89.31,50.0,3560.0,45.2,30.3,4.0


In [13]:
# CCI buy
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%'] > 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 2


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
5,2022-12-30 00:00:00+05:30,511.0,JUBLFOOD.NS,-8.25,-0.18,-1.44,785.0,463.0,14.92,85.08,37.0,643.0,87.9,71.2,26.0
9,2022-12-30 00:00:00+05:30,1100.0,HAVELLS.NS,-10.43,0.08,-1.17,1400.0,1068.0,9.62,90.38,27.0,1328.0,63.7,63.3,21.0


In [14]:
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df['Upside%']*100/tmp_df['Upside%'].sum()

1     19.090909
2     16.363636
3     14.090909
4     12.727273
10     9.090909
12     8.636364
18     6.363636
21     6.363636
29     5.454545
51     1.818182
Name: Upside%, dtype: float64

In [15]:
# heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%'] > 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 10


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
0,2022-12-30 00:00:00+05:30,430.0,GODREJIND.NS,-5.31,-0.88,0.00,644.0,405.0,10.42,89.58,40.0,638.0,39.5,18.7,48.0
5,2022-12-30 00:00:00+05:30,511.0,JUBLFOOD.NS,-8.25,-0.18,-1.44,785.0,463.0,14.92,85.08,37.0,643.0,87.9,71.2,26.0
6,2022-12-30 00:00:00+05:30,240.0,ZEEL.NS,-4.36,0.00,-1.16,325.0,206.0,28.61,71.39,34.0,303.0,24.6,28.5,26.0
7,2022-12-30 00:00:00+05:30,891.0,ICICIBANK.NS,9.44,1.23,0.22,953.0,650.0,79.49,20.51,42.0,1094.0,24.2,21.0,23.0
8,2022-12-30 00:00:00+05:30,6575.0,BAJFINANCE.NS,-1.57,-0.04,-1.19,7824.0,5265.0,51.20,48.80,47.0,8015.0,50.8,40.1,22.0
9,2022-12-30 00:00:00+05:30,1100.0,HAVELLS.NS,-10.43,0.08,-1.17,1400.0,1068.0,9.62,90.38,27.0,1328.0,63.7,63.3,21.0
11,2022-12-30 00:00:00+05:30,3228.0,EICHERMOT.NS,5.24,1.40,-0.93,3850.0,2241.0,61.33,38.67,43.0,3884.0,40.1,37.8,20.0
13,2022-12-30 00:00:00+05:30,1249.0,M&M.NS,11.34,2.23,0.00,1354.0,684.0,84.36,15.64,47.0,1468.0,20.5,18.6,18.0
14,2022-12-30 00:00:00+05:30,81.0,IBREALEST.NS,0.12,-1.23,1.22,174.0,60.0,18.51,81.49,48.0,95.0,11.3,1000.0,17.0
15,2022-12-30 00:00:00+05:30,614.0,SBIN.NS,16.90,1.33,1.17,626.0,434.0,93.59,6.41,57.0,710.0,13.2,13.3,16.0


In [16]:
# near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Discount%'] > 85]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Premium%', ascending=True)

qualified stocks: 6


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
42,2022-12-30 00:00:00+05:30,393.0,WIPRO.NS,-11.14,-2.04,0.25,714.0,375.0,5.24,94.76,50.0,425.0,19.0,19.1,8.0
9,2022-12-30 00:00:00+05:30,1100.0,HAVELLS.NS,-10.43,0.08,-1.17,1400.0,1068.0,9.62,90.38,27.0,1328.0,63.7,63.3,21.0
28,2022-12-30 00:00:00+05:30,581.0,BERGEPAINT.NS,-7.87,-0.48,-0.50,787.0,558.0,10.20,89.80,37.0,651.0,72.9,59.7,12.0
0,2022-12-30 00:00:00+05:30,430.0,GODREJIND.NS,-5.31,-0.88,0.00,644.0,405.0,10.42,89.58,40.0,638.0,39.5,18.7,48.0
27,2022-12-30 00:00:00+05:30,208.0,TATAPOWER.NS,-8.90,-0.44,-0.45,287.0,195.0,13.80,86.20,40.0,232.0,20.4,26.1,12.0
5,2022-12-30 00:00:00+05:30,511.0,JUBLFOOD.NS,-8.25,-0.18,-1.44,785.0,463.0,14.92,85.08,37.0,643.0,87.9,71.2,26.0


In [17]:
# near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Discount%'] < 10]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 9


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
60,2022-12-30 00:00:00+05:30,333.0,CANBK.NS,38.30,2.49,2.59,333.0,175.0,100.19,-0.19,61.0,322.0,8.8,6.45,-3.0
39,2022-12-30 00:00:00+05:30,139.0,FEDERALBNK.NS,26.41,1.82,0.75,139.0,81.0,100.09,-0.09,59.0,152.0,11.0,11.80,9.0
52,2022-12-30 00:00:00+05:30,113.0,TATASTEEL.NS,29.48,3.45,0.93,116.0,44.0,95.35,4.65,57.0,118.0,6.0,4.79,4.0
23,2022-12-30 00:00:00+05:30,934.0,AXISBANK.NS,21.74,1.43,1.33,950.0,625.0,95.00,5.00,58.0,1060.0,31.3,15.60,13.0
58,2022-12-30 00:00:00+05:30,186.0,BANKBARODA.NS,46.22,3.15,2.98,193.0,78.0,93.65,6.35,61.0,183.0,13.5,9.77,-2.0
15,2022-12-30 00:00:00+05:30,614.0,SBIN.NS,16.90,1.33,1.17,626.0,434.0,93.59,6.41,57.0,710.0,13.2,13.30,16.0
57,2022-12-30 00:00:00+05:30,285.0,INDIANB.NS,47.09,3.61,2.59,300.0,127.0,91.53,8.47,53.0,284.0,7.4,8.17,-0.0
30,2022-12-30 00:00:00+05:30,1628.0,HDFCBANK.NS,12.05,0.89,1.26,1662.0,1281.0,91.12,8.88,54.0,1813.0,26.0,21.80,11.0
22,2022-12-30 00:00:00+05:30,1220.0,INDUSINDBK.NS,18.11,1.55,0.09,1264.0,771.0,91.10,8.90,55.0,1393.0,22.8,15.60,14.0


In [18]:
# turning around (exhaustive)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > -.5) & (tmp_df['Slope%_200'] < 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 7


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
43,2022-12-30 00:00:00+05:30,1039.0,HCLTECH.NS,3.11,-0.40,0.47,1291.0,865.0,40.92,59.08,44.0,1117.0,16.3,20.4,8.0
53,2022-12-30 00:00:00+05:30,3257.0,TCS.NS,-1.25,-0.39,0.46,3965.0,2974.0,28.53,71.47,46.0,3385.0,28.1,30.1,4.0
50,2022-12-30 00:00:00+05:30,4069.0,DMART.NS,0.76,-0.17,-0.34,4731.0,3231.0,55.85,44.15,51.0,4282.0,138.7,114.0,5.0
27,2022-12-30 00:00:00+05:30,208.0,TATAPOWER.NS,-8.90,-0.44,-0.45,287.0,195.0,13.80,86.20,40.0,232.0,20.4,26.1,12.0
28,2022-12-30 00:00:00+05:30,581.0,BERGEPAINT.NS,-7.87,-0.48,-0.50,787.0,558.0,10.20,89.80,37.0,651.0,72.9,59.7,12.0
8,2022-12-30 00:00:00+05:30,6575.0,BAJFINANCE.NS,-1.57,-0.04,-1.19,7824.0,5265.0,51.20,48.80,47.0,8015.0,50.8,40.1,22.0
5,2022-12-30 00:00:00+05:30,511.0,JUBLFOOD.NS,-8.25,-0.18,-1.44,785.0,463.0,14.92,85.08,37.0,643.0,87.9,71.2,26.0


In [19]:
# turning around (compact)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 7


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
14,2022-12-30 00:00:00+05:30,81.0,IBREALEST.NS,0.12,-1.23,1.22,174.0,60.0,18.51,81.49,48.0,95.0,11.3,1000.00,17.0
25,2022-12-30 00:00:00+05:30,80.0,NATIONALUM.NS,-4.29,-2.38,1.33,128.0,67.0,21.97,78.03,60.0,90.0,10.3,5.82,12.0
37,2022-12-30 00:00:00+05:30,473.0,HINDALCO.NS,9.32,-1.39,1.81,626.0,314.0,51.07,48.93,58.0,514.0,10.4,7.67,9.0
42,2022-12-30 00:00:00+05:30,393.0,WIPRO.NS,-11.14,-2.04,0.25,714.0,375.0,5.24,94.76,50.0,425.0,19.0,19.10,8.0
43,2022-12-30 00:00:00+05:30,1039.0,HCLTECH.NS,3.11,-0.40,0.47,1291.0,865.0,40.92,59.08,44.0,1117.0,16.3,20.40,8.0
53,2022-12-30 00:00:00+05:30,3257.0,TCS.NS,-1.25,-0.39,0.46,3965.0,2974.0,28.53,71.47,46.0,3385.0,28.1,30.10,4.0
54,2022-12-30 00:00:00+05:30,83.0,SAIL.NS,2.04,-1.23,1.20,108.0,63.0,43.67,56.33,52.0,85.0,6.9,7.19,2.0


In [20]:
# bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 8


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
60,2022-12-30 00:00:00+05:30,333.0,CANBK.NS,38.30,2.49,2.59,333.0,175.0,100.19,-0.19,61.0,322.0,8.8,6.45,-3.0
41,2022-12-30 00:00:00+05:30,2008.0,INDIGO.NS,8.70,0.65,1.32,2265.0,1549.0,64.06,35.94,55.0,2176.0,30.7,1000.00,8.0
33,2022-12-30 00:00:00+05:30,2638.0,HDFC.NS,11.15,0.93,1.27,2729.0,2053.0,86.48,13.52,47.0,2940.0,23.0,19.90,11.0
30,2022-12-30 00:00:00+05:30,1628.0,HDFCBANK.NS,12.05,0.89,1.26,1662.0,1281.0,91.12,8.88,54.0,1813.0,26.0,21.80,11.0
45,2022-12-30 00:00:00+05:30,6959.0,ULTRACEMCO.NS,8.80,0.73,1.10,7825.0,5148.0,67.65,32.35,47.0,7420.0,33.7,30.20,7.0
34,2022-12-30 00:00:00+05:30,561.0,DABUR.NS,3.02,0.18,0.71,604.0,480.0,65.69,34.31,41.0,625.0,53.6,55.50,11.0
56,2022-12-30 00:00:00+05:30,878.0,MCDOWELL-N.NS,4.22,0.24,0.56,947.0,732.0,67.70,32.30,39.0,885.0,74.5,57.00,1.0
26,2022-12-30 00:00:00+05:30,2547.0,RELIANCE.NS,-0.34,0.43,0.46,2811.0,2229.0,54.67,45.33,43.0,2854.0,25.3,26.80,12.0


In [21]:
# about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 9


,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,RSI,Forecast,Med_PE,Curr_PE,Upside%
37,2022-12-30 00:00:00+05:30,473.0,HINDALCO.NS,9.32,-1.39,1.81,626.0,314.0,51.07,48.93,58.0,514.0,10.4,7.67,9.0
25,2022-12-30 00:00:00+05:30,80.0,NATIONALUM.NS,-4.29,-2.38,1.33,128.0,67.0,21.97,78.03,60.0,90.0,10.3,5.82,12.0
14,2022-12-30 00:00:00+05:30,81.0,IBREALEST.NS,0.12,-1.23,1.22,174.0,60.0,18.51,81.49,48.0,95.0,11.3,1000.00,17.0
54,2022-12-30 00:00:00+05:30,83.0,SAIL.NS,2.04,-1.23,1.20,108.0,63.0,43.67,56.33,52.0,85.0,6.9,7.19,2.0
43,2022-12-30 00:00:00+05:30,1039.0,HCLTECH.NS,3.11,-0.40,0.47,1291.0,865.0,40.92,59.08,44.0,1117.0,16.3,20.40,8.0
53,2022-12-30 00:00:00+05:30,3257.0,TCS.NS,-1.25,-0.39,0.46,3965.0,2974.0,28.53,71.47,46.0,3385.0,28.1,30.10,4.0
42,2022-12-30 00:00:00+05:30,393.0,WIPRO.NS,-11.14,-2.04,0.25,714.0,375.0,5.24,94.76,50.0,425.0,19.0,19.10,8.0
0,2022-12-30 00:00:00+05:30,430.0,GODREJIND.NS,-5.31,-0.88,0.00,644.0,405.0,10.42,89.58,40.0,638.0,39.5,18.70,48.0
44,2022-12-30 00:00:00+05:30,4365.0,LTIM.NS,-6.92,-1.79,-0.62,7506.0,3796.0,15.35,84.65,42.0,4654.0,26.6,50.40,7.0
